# Desafio 6

Neste desafio, vamos praticar _feature engineering_, um dos processos mais importantes e trabalhosos de ML. Utilizaremos o _data set_ [Countries of the world](https://www.kaggle.com/fernandol/countries-of-the-world), que contém dados sobre os 227 países do mundo com informações sobre tamanho da população, área, imigração e setores de produção.

> Obs.: Por favor, não modifique o nome das funções de resposta.

## _Setup_ geral

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn as sk
from sklearn.preprocessing import (
    OneHotEncoder, Binarizer, KBinsDiscretizer,
    MinMaxScaler, StandardScaler, PolynomialFeatures)
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

In [2]:
# Algumas configurações para o matplotlib.
#%matplotlib inline

#from IPython.core.pylabtools import figsize


#figsize(12, 8)

#sns.set()

In [3]:
countries = pd.read_csv("countries.csv")

In [4]:
new_column_names = [
    "Country", "Region", "Population", "Area", "Pop_density", "Coastline_ratio",
    "Net_migration", "Infant_mortality", "GDP", "Literacy", "Phones_per_1000",
    "Arable", "Crops", "Other", "Climate", "Birthrate", "Deathrate", "Agriculture",
    "Industry", "Service"
]

countries.columns = new_column_names

countries.head(5)

,Country,Region,Population,Area,Pop_density,Coastline_ratio,Net_migration,Infant_mortality,GDP,Literacy,Phones_per_1000,Arable,Crops,Other,Climate,Birthrate,Deathrate,Agriculture,Industry,Service
0,Afghanistan,ASIA (EX. NEAR EAST),31056997,647500,"48,0","0,00","23,06","163,07",700.0,"36,0","3,2","12,13","0,22","87,65",1,"46,6","20,34","0,38","0,24","0,38"
1,Albania,EASTERN EUROPE,3581655,28748,"124,6","1,26","-4,93","21,52",4500.0,"86,5","71,2","21,09","4,42","74,49",3,"15,11","5,22","0,232","0,188","0,579"
2,Algeria,NORTHERN AFRICA,32930091,2381740,"13,8","0,04","-0,39",31,6000.0,"70,0","78,1","3,22","0,25","96,53",1,"17,14","4,61","0,101","0,6","0,298"
3,American Samoa,OCEANIA,57794,199,"290,4","58,29","-20,71","9,27",8000.0,"97,0","259,5",10,15,75,2,"22,46","3,27",NaN,NaN,NaN
4,Andorra,WESTERN EUROPE,71201,468,"152,1","0,00","6,6","4,05",19000.0,"100,0","497,2","2,22",0,"97,78",3,"8,71","6,25",NaN,NaN,NaN


## Observações

Esse _data set_ ainda precisa de alguns ajustes iniciais. Primeiro, note que as variáveis numéricas estão usando vírgula como separador decimal e estão codificadas como strings. Corrija isso antes de continuar: transforme essas variáveis em numéricas adequadamente.

Além disso, as variáveis `Country` e `Region` possuem espaços a mais no começo e no final da string. Você pode utilizar o método `str.strip()` para remover esses espaços.

## Inicia sua análise a partir daqui

#### Organizando 

In [5]:
countries.dtypes

Country              object
Region               object
Population            int64
Area                  int64
Pop_density          object
Coastline_ratio      object
Net_migration        object
Infant_mortality     object
GDP                 float64
Literacy             object
Phones_per_1000      object
Arable               object
Crops                object
Other                object
Climate              object
Birthrate            object
Deathrate            object
Agriculture          object
Industry             object
Service              object
dtype: object

In [6]:
#Fazendo a troca das "," por "."
countries.iloc[:,[4,5,6,7,9,10,11,12,13,14,15,16,17,18,19]] = countries.iloc[:,[4,5,6,7,9,10,11,12,13,14,15,16,17,18,19]]\
                                                                    .apply(lambda x: x.str.replace(',' , '.'))

# Convertendo os elementos que são float para o tipo object
countries.iloc[:,[4,5,6,7,9,10,11,12,13,14,15,16,17,18,19]] = countries.iloc[:,[4,5,6,7,9,10,11,12,13,14,15,16,17,18,19]]\
                                                                    .astype(float)

In [7]:
countries.dtypes

Country              object
Region               object
Population            int64
Area                  int64
Pop_density         float64
Coastline_ratio     float64
Net_migration       float64
Infant_mortality    float64
GDP                 float64
Literacy            float64
Phones_per_1000     float64
Arable              float64
Crops               float64
Other               float64
Climate             float64
Birthrate           float64
Deathrate           float64
Agriculture         float64
Industry            float64
Service             float64
dtype: object

In [8]:
# Removendo espaços desnecessários das colunas Country e Region com a função str.strip
countries.loc[:,['Country', 'Region']] = countries.loc[:,['Country', 'Region']].applymap(str.strip)

## Questão 1

Quais são as regiões (variável `Region`) presentes no _data set_? Retorne uma lista com as regiões únicas do _data set_ com os espaços à frente e atrás da string removidos (mas mantenha pontuação: ponto, hífen etc) e ordenadas em ordem alfabética.

In [9]:
def q1():
    
    # Gerando a lista de regiões únicas
    lista = list(countries.Region.unique())
    
    # Ordenando a lista
    lista.sort()
    
    # Visualizando a lista
    return lista

In [10]:
q1()

['ASIA (EX. NEAR EAST)',
 'BALTICS',
 'C.W. OF IND. STATES',
 'EASTERN EUROPE',
 'LATIN AMER. & CARIB',
 'NEAR EAST',
 'NORTHERN AFRICA',
 'NORTHERN AMERICA',
 'OCEANIA',
 'SUB-SAHARAN AFRICA',
 'WESTERN EUROPE']

## Questão 2

Discretizando a variável `Pop_density` em 10 intervalos com `KBinsDiscretizer`, seguindo o encode `ordinal` e estratégia `quantile`, quantos países se encontram acima do 90º percentil? Responda como um único escalar inteiro.

In [139]:
def q2():
    #Aplicando a discretização
         #Formato requetido
    discretizer = KBinsDiscretizer(n_bins=10, encode="ordinal", strategy="quantile")
       
        #Ajustando o sistema para que aprenda esses valores
    discretizer.fit(countries[["Pop_density"]])

        #Fazendo as transformações para cada um dos intervalos e plotando
    ajuste = discretizer.transform(countries[["Pop_density"]])
    
    # Filtrando e contando valores maiores que o quartil
    qnt = int((ajuste > np.quantile(ajuste, 0.9)).sum())

    return qnt

In [140]:
q2()

23

# Questão 3

Se codificarmos as variáveis `Region` e `Climate` usando _one-hot encoding_, quantos novos atributos seriam criados? Responda como um único escalar.

In [164]:
def q3():
    
    # Removendo Valores NAN
    countries1 = countries.dropna()
        
    # Configurando o encoder
    encoder = OneHotEncoder(sparse=False, dtype=np.int, )
    
    # Aplicando o encoder nas colunas Region e Climate
    data_encoded = encoder.fit_transform(countries1[['Region','Climate']])
    
    return (data_encoded.shape[1]+1)

In [165]:
q3()

18

## Questão 4

Aplique o seguinte _pipeline_:

1. Preencha as variáveis do tipo `int64` e `float64` com suas respectivas medianas.
2. Padronize essas variáveis.

Após aplicado o _pipeline_ descrito acima aos dados (somente nas variáveis dos tipos especificados), aplique o mesmo _pipeline_ (ou `ColumnTransformer`) ao dado abaixo. Qual o valor da variável `Arable` após o _pipeline_? Responda como um único float arredondado para três casas decimais.

In [43]:
test_country = [
    'Test Country', 'NEAR EAST', -0.19032480757326514,
    -0.3232636124824411, -0.04421734470810142, -0.27528113360605316,
    0.13255850810281325, -0.8054845935643491, 1.0119784924248225,
    0.6189182532646624, 1.0074863283776458, 0.20239896852403538,
    -0.043678728558593366, -0.13929748680369286, 1.3163604645710438,
    -0.3699637766938669, -0.6149300604558857, -0.854369594993175,
    0.263445277972641, 0.5712416961268142
]

In [168]:
def q4():
   
    #Aplicando a sequencia
    num_pipeline = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())])
    
    # Treinando o pipeline para as colunas que possuem numemos(np.number)
    num_pipeline.fit(countries.select_dtypes([np.number]))
    
    #Obtendo os valores das medianas das colunas em questão
    medianas = num_pipeline.transform([test_country[2:]])
    
    return float(medianas[0][9].round(3))

In [169]:
q4()

-1.047

## Questão 5

Descubra o número de _outliers_ da variável `Net_migration` segundo o método do _boxplot_, ou seja, usando a lógica:

$$x \notin [Q1 - 1.5 \times \text{IQR}, Q3 + 1.5 \times \text{IQR}] \Rightarrow x \text{ é outlier}$$

que se encontram no grupo inferior e no grupo superior.

Você deveria remover da análise as observações consideradas _outliers_ segundo esse método? Responda como uma tupla de três elementos `(outliers_abaixo, outliers_acima, removeria?)` ((int, int, bool)).

In [141]:
def q5():
    
    aux = countries['Net_migration']
    
    q1 = aux.quantile(0.25)
    q3 = aux.quantile(0.75)
    iqr = q3 - q1

    faixa = [q1 - 1.5 * iqr, q3 + 1.5 * iqr]
    
    abaixo = int((aux < faixa[0]).sum())
    acima = int((aux > faixa[1]).sum())
    
    qnt_fora = abaixo + acima
    total = aux.shape[0]
    
    remover = bool( qnt_fora/total < 0.05 )
    
    return (abaixo,acima,remover)
    

In [142]:
q5()

(24, 26, False)

## Questão 6
Para as questões 6 e 7 utilize a biblioteca `fetch_20newsgroups` de datasets de test do `sklearn`

Considere carregar as seguintes categorias e o dataset `newsgroups`:

```
categories = ['sci.electronics', 'comp.graphics', 'rec.motorcycles']
newsgroup = fetch_20newsgroups(subset="train", categories=categories, shuffle=True, random_state=42)
```


Aplique `CountVectorizer` ao _data set_ `newsgroups` e descubra o número de vezes que a palavra _phone_ aparece no corpus. Responda como um único escalar.

In [128]:
from sklearn.datasets import fetch_20newsgroups 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [145]:
#Aplicando as categorias
categories = ['sci.electronics', 'comp.graphics', 'rec.motorcycles']
newsgroup = fetch_20newsgroups(subset="train", categories=categories, shuffle=True, random_state=42)
    

In [166]:
def q6():
    
    #Aplicando as condiçoes para a contagem 
    count_vectorizer = CountVectorizer()
    newsgroups_counts = count_vectorizer.fit_transform(newsgroup.data)
    
    return int(newsgroups_counts[:,count_vectorizer.vocabulary_['phone']].sum())
    

In [167]:
q6()

213

## Questão 7

Aplique `TfidfVectorizer` ao _data set_ `newsgroups` e descubra o TF-IDF da palavra _phone_. Responda como um único escalar arredondado para três casas decimais.

In [162]:
def q7():
    
    #Aplicando o TfidfVectorizer
    tfidf_vectorizer = TfidfVectorizer()

    #Ajustando os valores
    words = tfidf_vectorizer.fit_transform(newsgroup.data)


    return round(float(words[:, tfidf_vectorizer.vocabulary_['phone']].sum()),3)

In [163]:
q7()

8.888